In [1]:
import os
os.chdir(os.path.pardir)
from dataset.dataset import Dataset
from evaluation_metrics.diversity_metrics import Topic_diversity
from evaluation_metrics.topic_significance_metrics import KL_uniform
from skopt import gp_minimize, forest_minimize, dummy_minimize
from optimization.optimizer import Optimizer
from skopt.space.space import Real, Integer
import multiprocessing as mp
from models import TorchETM
import torch
import numpy as np

In [2]:
dataset = Dataset()
dataset.load("preprocessed_datasets/newsgroup/newsgroup_lemmatized_10")

True

In [3]:
# Load model
model = TorchETM.ETM_Wrapper()

In [4]:
model.hyperparameters['epochs'] = 5
model.hyperparameters['enc_drop'] = 0.1

In [5]:
model.test_set(False)

In [6]:
model.train_model(dataset, model.hyperparameters, top_words= 5)


model: ETM(
  (t_drop): Dropout(p=0.1, inplace=False)
  (theta_act): ReLU()
  (rho): Linear(in_features=300, out_features=1268, bias=False)
  (alphas): Linear(in_features=300, out_features=10, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=1268, out_features=800, bias=True)
    (1): ReLU()
    (2): Linear(in_features=800, out_features=800, bias=True)
    (3): ReLU()
  )
  (mu_q_theta): Linear(in_features=800, out_features=10, bias=True)
  (logsigma_q_theta): Linear(in_features=800, out_features=10, bias=True)
)
****************************************************************************************************
Epoch----->0 .. LR: 0.005 .. KL_theta: 0.04 .. Rec_loss: 502.39 .. NELBO: 502.43
****************************************************************************************************
****************************************************************************************************
Epoch----->1 .. LR: 0.005 .. KL_theta: 0.65 .. Rec_loss: 497.05 .. NELBO: 497.7
*

{'topics': [['turn', 'victim', 'playoff', 'thread', 'library'],
  ['turn', 'playoff', 'victim', 'thread', 'guess'],
  ['son', 'shipping', 'screw', 'security', 'mostly'],
  ['victim', 'playoff', 'method', 'library', 'past'],
  ['turn', 'playoff', 'victim', 'thread', 'library'],
  ['victim', 'playoff', 'turn', 'library', 'thread'],
  ['victim', 'playoff', 'turn', 'library', 'thread'],
  ['generation', 'card', 'license', 'hope', 'victim'],
  ['encryption', 'school', 'manner', 'base', 'basis'],
  ['military', 'method', 'generation', 'food', 'bunch']],
 'topic-word-matrix': array([[8.4459745e-05, 1.8019975e-03, 1.9959357e-04, ..., 1.4792911e-04,
         1.0343851e-03, 7.7922530e-05],
        [9.0886628e-05, 1.9206755e-03, 2.3516656e-04, ..., 1.8121021e-04,
         1.1289663e-03, 9.2179056e-05],
        [3.1988561e-04, 1.1765852e-03, 4.9168520e-05, ..., 2.6914446e-05,
         3.0845561e-04, 5.1107359e-05],
        ...,
        [1.7030124e-04, 7.2725926e-04, 4.6173940e-04, ..., 4.7927702e-